# TF-IDF를 활용한 Passage Retrieval 실습

### Requirements

In [5]:
# > /dev/null 2>&1 # execute command in silence
!pip install datasets==2.4.0 > /dev/null 2>&1 # execute command in silence
!pip install transformers==4.20.1 > /dev/null 2>&1

������ ��θ� ã�� �� �����ϴ�.
������ ��θ� ã�� �� �����ϴ�.


## 데이터셋 준비

KorQuAD train 데이터셋을 search corpus로 활용

In [6]:
from datasets import load_dataset

dataset = load_dataset("squad_kor_v1")

Reusing dataset squad_kor_v1 (C:\Users\AI_15\.cache\huggingface\datasets\squad_kor_v1\squad_kor_v1\1.0.0\18d4f44736b8ee85671f63cb84965bfb583fa0a4ff2df3c2e10eee9693796725)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
corpus = list(set([example['context'] for example in dataset['train']]))
len(corpus)

9606

## 토크나이저 준비

가장 기본적인 띄워쓰기를 기준으로 token을 나누는 tokenizer를 활용 (성능 향상을 위해 더 세밀한 토크나이저 활용 가능)

In [4]:
tokenizer_func = lambda x: x.split(' ')

In [7]:
tokenizer_func(corpus[0])[:10]

['미국인은', '과소비성', '소비행태로', '알려져있으나,', '사실은', '소비', '행동이', '절약적으로,', '소량', '구매,']

## TF-IDF embedding 만들기

Scikit-learn의 TfidfVectorizer를 활용하여 TF-IDF embedding 만들어보기 
(unigram, bigram 활용)

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(tokenizer=tokenizer_func, ngram_range=(1,2))

In [9]:
vectorizer.fit(corpus)
sp_matrix = vectorizer.transform(corpus)

c:\Users\AI_15\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\feature_extraction\text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [10]:
sp_matrix.shape

(9606, 1272768)

In [11]:
import pandas as pd
df = pd.DataFrame(sp_matrix[0].T.todense(), index=vectorizer.get_feature_names(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
print(df.head(10))

c:\Users\AI_15\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


             TF-IDF
소비         0.148038
세계에서 가장    0.115786
문화         0.111599
세계에서       0.101285
이유로 미국은    0.085294
소비행태로      0.085294
구입하는 나라며,  0.085294
구매, 쿠폰의    0.085294
살며, 빈부격차는  0.085294
가장 부자가     0.085294


## TF-IDF embedding을 활용하여 passage retrieval 실습해보기

앞서 만든 sparse embedding을 활용하여, 실제 passage retrieval 수행해보기

Search query로 KorQuAD train 데이터셋의 질문을 활용하여, 실제 context가 잘 나오는지 확인해보기

In [12]:
import random
import numpy as np

random.seed(1)
sample_idx = random.choice(range(len(dataset['train'])))

query = dataset['train'][sample_idx]['question']
ground_truth = dataset['train'][sample_idx]['context']

Query를 tf-idf vector로 변환

In [13]:
query_vec = vectorizer.transform([query])

In [14]:
query_vec.shape

(1, 1272768)

변환된 query vector를 document들의 vector과 dot product를 수행 => Document들의 similarity ranking을 구함

In [15]:
result = query_vec * sp_matrix.T
result.shape

(1, 9606)

In [16]:
sorted_result = np.argsort(-result.data)
doc_scores = result.data[sorted_result]
doc_ids = result.indices[sorted_result]

Top-3개의 passage를 retrieve 하고, 실제 ground_truth와 비교해보기

In [17]:
k = 3
doc_scores[:k], doc_ids[:k]

(array([0.18985967, 0.03625019, 0.03371167]),
 array([9346, 5528, 4125], dtype=int32))

In [18]:
print("[Search query]\n", query, "\n")

print("[Ground truth passage]")
print(ground_truth, "\n")

for i in range(k):
  print("Top-%d passage with score %.4f" % (i + 1, doc_scores[i]))
  doc_id = doc_ids[i]
  print(corpus[doc_id], "\n")


[Search query]
 호메로스 찬가를 신통기에 비해 간결한 서사로 간주한 사람은 누구인가? 

[Ground truth passage]
고전 시대 신화에서는 티탄들의 패배 이후, 신들의 새로운 판테온이 세워졌다고 설명한다. 주요한 그리스 신들 중에서 올림피안은 올림포스 산 정상에서 제우스의 통치 아래 살아가는 신들을 말한다. 이들의 인원이 열두 명으로 제한된 것은 비교적 최근에 도입된 개념으로 보인다. 올림피안 이외에도 그리스인들은 염소 신 판, 강의 정령 님프, 샘에 사는 나이아드, 나무의 정령 드라이어드, 바다에 사는 네레이드, 강의 신, 사티로스를 비롯한 그 지역의 다양한 신들을 숭배하였다. 여기에는 에리니에스(또는 푸리아이)처럼 혈연 관계에게 범죄를 저지른 죄인을 뒤쫓는 저승의 암흑 세력도 있었다. 시인들은 그리스 판테온의 영광을 기리고자 호메로스 찬가를 지었다.(33편의 노래). 그레고리 나지는 호메로스 찬가를 "각 노래마다 신에 대한 기원을 노래하는(《신통기》에 비해) 간결한 서가"로 간주하였다. 

Top-1 passage with score 0.1899
고전 시대 신화에서는 티탄들의 패배 이후, 신들의 새로운 판테온이 세워졌다고 설명한다. 주요한 그리스 신들 중에서 올림피안은 올림포스 산 정상에서 제우스의 통치 아래 살아가는 신들을 말한다. 이들의 인원이 열두 명으로 제한된 것은 비교적 최근에 도입된 개념으로 보인다. 올림피안 이외에도 그리스인들은 염소 신 판, 강의 정령 님프, 샘에 사는 나이아드, 나무의 정령 드라이어드, 바다에 사는 네레이드, 강의 신, 사티로스를 비롯한 그 지역의 다양한 신들을 숭배하였다. 여기에는 에리니에스(또는 푸리아이)처럼 혈연 관계에게 범죄를 저지른 죄인을 뒤쫓는 저승의 암흑 세력도 있었다. 시인들은 그리스 판테온의 영광을 기리고자 호메로스 찬가를 지었다.(33편의 노래). 그레고리 나지는 호메로스 찬가를 "각 노래마다 신에 대한 기원을 노래하는(《신통기》에 비해) 간결한 서가"로 간주하였다. 

Top-2 

## Wikipedia documents에 대해 TF-IDF 실습하기

In [19]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1wVIgtc0YoQEwXB3JAsUud_86fRzrFCBd' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1wVIgtc0YoQEwXB3JAsUud_86fRzrFCBd" -O wikipedia_documents.json && rm -rf /tmp/cookies.txt

Cannot open cookies file '/tmp/cookies.txt': No such file or directory
--2022-07-27 15:37:30--  https://docs.google.com/uc?export=download&confirm=$(wget%20--quiet%20--save-cookies%20/tmp/cookies.txt%20--keep-session-cookies%20--no-check-certificate%20'https://docs.google.com/uc?export=download&id=1wVIgtc0YoQEwXB3JAsUud_86fRzrFCBd'%20-O-%20%7C%20sed%20-rn%20's/.*confirm=([0-9A-Za-z_]+).*/%5C1%5Cn/p')&id=1wVIgtc0YoQEwXB3JAsUud_86fRzrFCBd
Resolving docs.google.com (docs.google.com)... 142.250.76.142
Connecting to docs.google.com (docs.google.com)|142.250.76.142|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2022-07-27 15:37:31 ERROR 404: Not Found.



In [24]:
# First load wikipedia dump
import json

# TODO: Write your own path
dump_path = 'wikipedia_documents.json'
with open(dump_path, 'r') as f:
    wiki = json.load(f)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
len(wiki), type(wiki)

(60613, dict)

In [ ]:
wiki['0']

{'author': None,
 'corpus_source': '위키피디아',
 'document_id': 0,
 'domain': None,
 'html': None,
 'text': '이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n\n두 목록은 모두 가나다 순이다.\n\n일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.',
 'title': '나라 목록',
 'url': 'TODO'}

In [ ]:
# TODO: Generate corpus and Train wiki_vectorizer on wikipedia_documents.json
corpus = [wiki[key]['text'] for key in wiki.keys()]
wiki_vectorizer = TfidfVectorizer(tokenizer=tokenizer_func, ngram_range=(1,2))
wiki_vectorizer.fit(corpus)
wiki_matrix = wiki_vectorizer.transform(corpus)


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


In [ ]:
# Answer: (60613, 8719114) (num_doc, num_entity)
wiki_matrix.shape

(60613, 8719114)

In [ ]:
query = '대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?'
query_vec = wiki_vectorizer.transform([query])

In [ ]:
result = query_vec * wiki_matrix.T
result.shape

(1, 60613)

In [ ]:
k = 3
doc_scores[:k], doc_ids[:k]

(array([0.18985967, 0.03625019, 0.03371167]),
 array([4583,  200, 3462], dtype=int32))

In [ ]:
print("[Search query]\n", query, "\n")


for i in range(k):
    print("Top-%d passage with score %.4f" % (i + 1, doc_scores[i]))
    doc_id = doc_ids[i]
    print(corpus[doc_id], "\n")

[Search query]
 대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은? 

Top-1 passage with score 0.1899
대수학(代數學, Algebra)은 일련의 공리들을 만족하는 수학적 구조들의 일반적인 성질을 연구하는 수학의 한 분야이다. 이렇게 일련의 추상적인 성질들로 정의되는 구조들을 대수 구조라고 하며, 그 예시로 반군, 군, 환, 가군, 체, 벡터 공간, 격자 등이 있다. 대수학은 취급하는 구조에 따라서 반군론, 군론, 환론, 선형대수학, 격자론, 정수론 등으로 분류된다.

기하학, 해석학, 정수론과 함께 대수학은 수학의 대분야 중 하나로 볼 수 있다. 대수학이란 용어는 단순한 산술적 수학을 가리키기도 하나, 수학자들은 군, 환, 불변량 이론과 같이 수 체계 및 그 체계 내에서의 연산에 대한 추상적 연구에 대해서 "대수학"이라는 용어를 자주 사용한다. 

Top-2 passage with score 0.0363
macOS(맥오에스, 이전 이름: OS X, 맥 OS X / Mac OS X )는 애플이 개발한 유닉스 운영 체제이다. 2002년 4월부터 모든 매킨토시 컴퓨터에 적용되고 있다. 이 운영 체제는 1984년 1월부터 애플 컴퓨터를 이끌어 왔던 맥 OS의 마지막 고전 버전인 맥 오에스 9의 뒤를 잇는다. OS X이라는 이 운영 체제의 예전 이름에 들어있는 "X"라는 글자는 알파벳 "X"을 뜻하는 것이 아니라, 매킨토시의 10번째 운영 체제를 뜻하는 것이기 때문에, 로마 숫자 "10"을 뜻하는 것이다.  이 운영 체제는 애플이 1996년 12월에 인수한  NeXT의 기술력으로 만들어졌으며 유닉스에 기반을 하고 있다.  2011년 7월 20일에 OS X 라이언이 출시되었다. 기존에도 줄여서 OS X이라고 많이 표현했으나, OS X 마운틴 라이언 공개와 함께 기존 맥 OS X (Mac OS X)에서 맥 (Mac)이라는 단어가 공식적으로 제거되었다.

2016년 6월 13일에 macOS 시에라 공개와 함께 제품명이 OS X에서